#### Execute L23

In [20]:
from Crypto.Util.Padding import pad, unpad
from Crypto.Cipher import AES
from hashlib import sha256

from qkd import QKD

bits = 32
qkd = QKD(bits)

alice_raw_key, bob_raw_key = qkd.generate_raw_key()
print(f"Alice raw key: {alice_raw_key}")
print(f"Bob raw key: {bob_raw_key}")

alice_key, bob_key, lists = qkd.generate_sifting_key(alice_raw_key, bob_raw_key)

assert alice_key == bob_key
print("QKD completed!")

print(f"Shared key: {alice_key}")

alice_key = sha256(alice_key.encode()).digest()
bob_key = sha256(bob_key.encode()).digest()

MSG = b"H3xTEL{R3v3rS3_R3C0nc1l14710N_1s_P34K!}"
cipher = AES.new(alice_key, AES.MODE_ECB)
encrypted = cipher.encrypt(pad(MSG, 16))

print(f"Alice encrypted message: {encrypted.hex()}")

cipher = AES.new(bob_key, AES.MODE_ECB)
decrypted = unpad(cipher.decrypt(encrypted), 16)

print(f"Bob decrypted message: {decrypted.decode()}")

Alice raw key: 11001000001010101100001111110001
Bob raw key: 00001100001010101101001011010001
QKD completed!
Shared key: 011011001010110111011011010110101000100000100011110011100110010101000000001000000100001100111111101000000101011000111100100100110111100010100000100111111100010101111101000001011011101101010011010101010100000101100001011010000000001110111100001101011011001111000001100001011111000000100111010110001011010010100101010100011101010110101110010110101011100100011010011101110011111001000100001110001111011000100101000111101001111001010101110001101001110111100101001010111011101110000100011001100110100111011000110000010001000010010000100101011000011000111000001010100101010110000110111001111000101010010001010001011011001011110110010001000001010000001001100101001011001111011001000010100001101001011111111010010100100001011110111110100100011110001010000111110011001001000101000111111110010011100101101000010010010111111011110111000001110011010101110101011011100101001010100101000111111

In [21]:
from key_recovery import attack
from itertools import product

key_recovered = []
for L1, L2 in zip(lists["L1"], lists["L2"]):
    rec = attack(L1, L2)
    key_recovered.append(rec)
    rec = rec.replace('1', '2').replace('0', '1').replace('2', '0')
    key_recovered.append(rec)

for x, y in product(key_recovered, key_recovered):
    key = x + y
    key = sha256(key.encode()).digest()
    if key == bob_key:
        print("Success!")
        break

Success!
